In [1]:
import pandas as pd
import numpy as np
import glob
import sys
import os

import matplotlib.pyplot as plt
import seaborn as sns

import importlib

root_dir = '/oak/stanford/groups/horence/rob/isoform_localizations/'

sys.path.append(os.path.join(root_dir,'scripts'))
import spatial_utils
import plot_utils

In [2]:
spots = pd.read_csv('../processed_data/all_passing_rna_spots.csv')
cells = pd.read_csv('../processed_data/segmented_cell_shapes.csv')

In [3]:
df = pd.read_csv('../sbatch_scripts/20210115_centrality.csv')
df.head()

,cell_id,target_molecule_name,num_cell_spots,num_gene_spots,metric_name,raw_metric,cell_zscore,gene_median_cell_zscore,gene_var_cell_zscore,spatial_utils_code_version
0,10000143038275111136124942858811168393,Adra1b,161,1,centrality,2.779945,-0.718970,0.061207,1.966323,b2a74e7616ec896e1a92651a30a1c06768c5feb71bba2c...
1,10000143038275111136124942858811168393,Aqp4,161,1,centrality,3.808437,0.005983,0.192908,3.527979,b2a74e7616ec896e1a92651a30a1c06768c5feb71bba2c...
2,10000143038275111136124942858811168393,Bcl11b,161,1,centrality,5.658580,1.310094,0.066685,12.304966,b2a74e7616ec896e1a92651a30a1c06768c5feb71bba2c...
3,10000143038275111136124942858811168393,Brinp3,161,1,centrality,4.015005,0.151587,0.000000,0.663956,b2a74e7616ec896e1a92651a30a1c06768c5feb71bba2c...
4,10000143038275111136124942858811168393,Cdh20,161,3,centrality,3.733510,-0.046831,-0.009072,0.747625,b2a74e7616ec896e1a92651a30a1c06768c5feb71bba2c...


In [4]:
quants = pd.read_csv('../processed_data/gene_per_cell_quantiles.csv')
quants.head()

,target_molecule_name,q10,q20,q30,q40,q50,q60,q70,q80,q90,q100
0,1700022I11Rik,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0
1,1810046K07Rik,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,6.0
2,5031425F14Rik,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0
3,5730522E02Rik,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0
4,Acta2,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,4.0,275.0


In [11]:
transmembrane_genes = [
    'Tmem163',
    'Tenm3',
    'Tmtc2',
    'Ssertm1',
    'Flrt3',
]

quantile = 'q90'
qs = quants[quants['target_molecule_name'].isin(transmembrane_genes)][['target_molecule_name',quantile]]
qs

,target_molecule_name,q90
93,Flrt3,7.0
227,Tenm3,15.0
232,Tmem163,12.0
233,Tmtc2,4.0


In [15]:
cells_of_interest = (
    spots[
        spots['target_molecule_name']
        .isin(transmembrane_genes)]
    .groupby(['cell_id','target_molecule_name'])
    .filter(lambda g: g.size > 4)
    [['cell_id','target_molecule_name']]
)

cells_of_interest

,cell_id,target_molecule_name
480,127525817974501757054778569557905409209,Flrt3
481,88185404565464270717298543203439325454,Flrt3
482,315730976604913875969464093634612175309,Flrt3
483,50895563111589999445359623445409082636,Flrt3
484,50895563111589999445359623445409082636,Flrt3
...,...,...
75825666,227747800804126471890839008344571017434,Tenm3
75825667,31877374488608324235206000954336084048,Tenm3
75825668,227747800804126471890839008344571017434,Tenm3
75825669,165661477580962561625505188705200110966,Tenm3


In [17]:
filt_counts = (
    spots[
        spots['target_molecule_name'].isin(transmembrane_genes) &
        spots['cell_id'].isin(cells_of_interest['cell_id'])
    ]
    .groupby(['cell_id','target_molecule_name'])
    .size()
)

filt_counts

cell_id                                  target_molecule_name
10000143038275111136124942858811168393   Tenm3                    3
100013893144618144270850756017896167367  Flrt3                    2
                                         Tenm3                    1
100017533667894826939857215877082515137  Tmtc2                    1
100019180101435853096549957382098546223  Flrt3                    2
                                                                 ..
99989592830367590092304100078674096866   Tmem163                  9
99997421766159526763299676887100858104   Flrt3                    1
                                         Tenm3                    1
                                         Tmem163                  1
99997470167777465101376150817407669505   Flrt3                   16
Length: 377764, dtype: int64

In [19]:
filt_counts = filt_counts.reset_index().rename(columns={0:'num_gene_counts'})
filt_counts

,cell_id,target_molecule_name,num_gene_counts
0,10000143038275111136124942858811168393,Tenm3,3
1,100013893144618144270850756017896167367,Flrt3,2
2,100013893144618144270850756017896167367,Tenm3,1
3,100017533667894826939857215877082515137,Tmtc2,1
4,100019180101435853096549957382098546223,Flrt3,2
...,...,...,...
377759,99989592830367590092304100078674096866,Tmem163,9
377760,99997421766159526763299676887100858104,Flrt3,1
377761,99997421766159526763299676887100858104,Tenm3,1
377762,99997421766159526763299676887100858104,Tmem163,1


In [20]:
qs

,target_molecule_name,q90
93,Flrt3,7.0
227,Tenm3,15.0
232,Tmem163,12.0
233,Tmtc2,4.0


In [21]:
filt_counts[quantile] = filt_counts['target_molecule_name'].map(qs.set_index('target_molecule_name')[quantile])
filt_counts

,cell_id,target_molecule_name,num_gene_counts,q90
0,10000143038275111136124942858811168393,Tenm3,3,15.0
1,100013893144618144270850756017896167367,Flrt3,2,7.0
2,100013893144618144270850756017896167367,Tenm3,1,15.0
3,100017533667894826939857215877082515137,Tmtc2,1,4.0
4,100019180101435853096549957382098546223,Flrt3,2,7.0
...,...,...,...,...
377759,99989592830367590092304100078674096866,Tmem163,9,12.0
377760,99997421766159526763299676887100858104,Flrt3,1,7.0
377761,99997421766159526763299676887100858104,Tenm3,1,15.0
377762,99997421766159526763299676887100858104,Tmem163,1,12.0


In [23]:
filt_counts = filt_counts[filt_counts['num_gene_counts'] >= filt_counts[quantile]]
filt_counts

,cell_id,target_molecule_name,num_gene_counts,q90
25,100041328926932918570941555647599217925,Tenm3,22,15.0
27,100042292675978612289781789372249900646,Tenm3,41,15.0
30,100043550028304404965222542799811957446,Tenm3,22,15.0
35,100046257586922188758017979350671101248,Tmtc2,4,4.0
40,100052143679862586867258825889400476028,Tmtc2,4,4.0
...,...,...,...,...
377739,99969166454721663399298872433695962292,Flrt3,11,7.0
377743,99972863828335872309426803572089317268,Tmtc2,6,4.0
377749,99983502058944096412442332908885097416,Tenm3,29,15.0
377751,99985816691327783828891364793469425625,Flrt3,15,7.0


In [34]:
import importlib
importlib.reload(plot_utils)
def plot_cell_helper(cell_id, gene_of_interest=None):
    spot_colors = {}
    if gene_of_interest != None:
        spot_colors = {gene_of_interest:'r'}
        

    plot_cells = cells[cells['cell_id'].eq(cell_id)]
    plot_spots = spots[spots['cell_id'].eq(cell_id)]
    fig,ax = plot_utils.plot_spot_cells(plot_spots,plot_cells,spot_colors)
    return fig,ax
    

In [38]:
import matplotlib.backends.backend_pdf
import importlib
importlib.reload(plot_utils)

gene = 'Flrt3'
pdf = matplotlib.backends.backend_pdf.PdfPages('{}_output.pdf'.format(gene))

i = 0
for _,r in filt_counts[filt_counts['target_molecule_name'].eq(gene)].iterrows():
    print(r)
    
    fig,ax = plot_cell_helper(r['cell_id'], gene_of_interest=r['target_molecule_name'])
    plt.title('Gene {} has {} counts in this cell, and q90 of {}'.format(
        gene, r['num_gene_counts'], r['q90'],
    ))
    pdf.savefig(fig)
    
    if i > 100:
        break
        
    i += 1
    
pdf.close()

cell_id                 100091009396667473995136644963828192189
target_molecule_name                                      Flrt3
num_gene_counts                                               7
q90                                                           7
Name: 90, dtype: object
cell_id                 100106378211286780818879112257146169937
target_molecule_name                                      Flrt3
num_gene_counts                                               9
q90                                                           7
Name: 102, dtype: object
cell_id                 100130021246024550058891566536662366819
target_molecule_name                                      Flrt3
num_gene_counts                                               7
q90                                                           7
Name: 144, dtype: object
cell_id                 10013848917901580372984707123288841246
target_molecule_name                                     Flrt3
num_gene_counts                 

/oak/stanford/groups/horence/rob/isoform_localizations/scripts/plot_utils.py:67: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(8,8))


cell_id                 1002898283446010279233117965444068592
target_molecule_name                                    Flrt3
num_gene_counts                                            27
q90                                                         7
Name: 338, dtype: object
cell_id                 100333151675726512762263903895381259981
target_molecule_name                                      Flrt3
num_gene_counts                                               9
q90                                                           7
Name: 398, dtype: object
cell_id                 100340335819132181432875565628414879129
target_molecule_name                                      Flrt3
num_gene_counts                                               8
q90                                                           7
Name: 409, dtype: object
cell_id                 100423572899316524273588760411709621929
target_molecule_name                                      Flrt3
num_gene_counts                      

cell_id                 101120573859982847364539472903580315206
target_molecule_name                                      Flrt3
num_gene_counts                                               8
q90                                                           7
Name: 1411, dtype: object
cell_id                 101297892386611843497429428213184719612
target_molecule_name                                      Flrt3
num_gene_counts                                               9
q90                                                           7
Name: 1689, dtype: object
cell_id                 101352046618284131518057764741436319530
target_molecule_name                                      Flrt3
num_gene_counts                                              14
q90                                                           7
Name: 1777, dtype: object
cell_id                 101396820644665977292320139615499466041
target_molecule_name                                      Flrt3
num_gene_counts           

cell_id                 101964371514237265308320690054940411790
target_molecule_name                                      Flrt3
num_gene_counts                                               8
q90                                                           7
Name: 2641, dtype: object
cell_id                 102010725846658874536430728824117450869
target_molecule_name                                      Flrt3
num_gene_counts                                               9
q90                                                           7
Name: 2698, dtype: object
cell_id                 102041908126107411729047924032229200917
target_molecule_name                                      Flrt3
num_gene_counts                                               7
q90                                                           7
Name: 2739, dtype: object
cell_id                 102048618169052974751363625075949587914
target_molecule_name                                      Flrt3
num_gene_counts           

cell_id                 102896590416167295283190430520914350200
target_molecule_name                                      Flrt3
num_gene_counts                                              12
q90                                                           7
Name: 3732, dtype: object
cell_id                 102941160209936048337208072650317826469
target_molecule_name                                      Flrt3
num_gene_counts                                              15
q90                                                           7
Name: 3764, dtype: object
cell_id                 102957237149353158963545122371009133807
target_molecule_name                                      Flrt3
num_gene_counts                                              11
q90                                                           7
Name: 3796, dtype: object
cell_id                 102969953952171517119598923156210439615
target_molecule_name                                      Flrt3
num_gene_counts           